In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_베이시스','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [2]:
df["1Y_베이시스대비"] =df["1Y_베이시스"]-df["1Y_베이시스"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_베이시스,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_베이시스대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,
2012-08-01,1126.5,-76.0,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,-74.0,2.79,2.77,1134.00,2.5,2.0,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,-72.0,2.77,2.75,1138.00,2.4,2.0,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,-74.0,2.78,2.76,1128.50,2.4,-2.0,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,-73.0,2.77,2.75,1129.00,2.5,1.0,-0.01,-0.01,0.50,0.1


In [3]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns

Index(['종가_ex', '1Y_베이시스', '국고1년', '통안364일', 'Mid_ndf', 'M1_스왑포인트',
       '1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비',
       '전날 종가_ex'],
      dtype='object')

In [5]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44736\4034184235.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_베이시스', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'], dtype='object')

In [17]:
# 스케일링 코드
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [19]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.917e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        21:19:42   Log-Likelihood:                -7749.4
No. Observations:                2459   AIC:                         1.551e+04
Df Residuals:                    2452   BIC:                         1.555e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        996.2660      3.501    284.597      0.000     989.402    1003.130
1Y_베이시스       -3.5516      1.392     -2.552      0.011      -6.280      -0.823
국고1년대비         2.9464      1.702      1.731      0.084      -0.392       6.285
통안364일대비       0.0252      3.716      0.007      0.995      -7.262       7.312
Mid_ndf대비     29.6770      5.453      5.442      0.000      18.983      40.371
M1_스왑포인트      -0.8813      0.782     -1.127      0.260      -2.415       0.653
전날 종가_ex     315.9689      0.709    445.721      0.000     314.579     317.359
==============================================================================
Omnibus:                      146.528   Durbin-Watson:                   2.164
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              621.790
Skew:                          -0.001   Prob(JB):                    9.55e-136
Kurtosis:                       5.463   Cond. No.                         90.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_베이시스', '국고1년대비', 'Mid_ndf대비','전날 종가_ex']]

In [21]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_scaled, has_constant='add')
# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 5.877e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        21:19:43   Log-Likelihood:                -7750.0
No. Observations:                2459   AIC:                         1.551e+04
Df Residuals:                    2454   BIC:                         1.554e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        995.5574      2.973    334.913      0.000     989.728    1001.386
1Y_베이시스       -3.5584      1.391     -2.558      0.011      -6.287      -0.830
국고1년대비         2.9083      1.683      1.728      0.084      -0.392       6.209
Mid_ndf대비     29.6857      5.453      5.444      0.000      18.994      40.378
전날 종가_ex     316.2714      0.656    482.138      0.000     314.985     317.558
==============================================================================
Omnibus:                      144.743   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              607.215
Skew:                           0.004   Prob(JB):                    1.40e-132
Kurtosis:                       5.434   Cond. No.                         76.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train, y_test = train_test_split(feature_add, df_Target,test_size=0.2, random_state=15 )

lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse

5.8690389674926235

In [24]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, feature_add, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.297641839574538,
 array([ 4.40558456,  6.73615954,  5.66429184, 19.48646447,  5.19570879]),
 array([ -19.40917531,  -45.37584535,  -32.08420201, -379.72229759,
         -26.99538984]))